In [42]:
#reading the csv files
import pandas as pd
from pathlib import Path
school_datacsv=Path("D:\pandas-challenge\Starter_Code\PyCitySchools\Resources\schools_complete.csv")
student_datacsv=Path("D:\pandas-challenge\Starter_Code\PyCitySchools\Resources\students_complete.csv")
school_data=pd.read_csv(school_datacsv)
student_data=pd.read_csv(student_datacsv)
combined_data=pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
combined_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [43]:
#district summary 
school_count=len(combined_data["school_name"].unique())
student_count=len(combined_data["Student ID"].unique())
total_budget=school_data["budget"].sum()
average_math_score=combined_data["math_score"].sum()/student_count
average_reading_score=combined_data["reading_score"].mean()
passing_math_count = combined_data[(combined_data["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_reading_count = combined_data[(combined_data["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_math_reading_count = combined_data[
    (combined_data["math_score"] >= 70) & (combined_data["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
print(overall_passing_rate)
print(passing_reading_percentage)
print(passing_math_percentage)
print(school_count)
print(student_count)
print(total_budget)
print(average_math_score)
print(average_reading_score)


65.17232575950983
85.80546336482001
74.9808526933878
15
39170
24649428
78.98537145774827
81.87784018381414


In [47]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = [{"Total Schools":school_count,
    "Total Students":student_count,
    "Total Budget":total_budget,
    "Average Math Score":average_math_score,
    "Average Reading Sscore":average_reading_score,
    "% Passing Math":passing_math_percentage,
    "% Passing Reading":passing_reading_percentage,
    "Overall Passing":overall_passing_rate}]
                     

#### Formatting
district_summary = pd.DataFrame({
    "Total Schools": school_count,
    "Total Students": f"{student_count:,}",
    "Total Budget": f"${total_budget:,.2f}",
    "Average Math Score": f"{average_math_score:.6f}",
    "Average Reading Score": f"{average_reading_score:.5f}",
    "% Passing Math": f"{passing_math_percentage:.6f}",
    "% Passing Reading": f"{passing_reading_percentage:.6f}",
    "Overall Passing": f"{overall_passing_rate: .6f}"
}, index=[0])
# Display the DataFrame
print(district_summary)

   Total Schools Total Students    Total Budget Average Math Score  \
0             15         39,170  $24,649,428.00          78.985371   

  Average Reading Score % Passing Math % Passing Reading Overall Passing  
0              81.87784      74.980853         85.805463       65.172326  


In [56]:
#school summary
school_name = combined_data.set_index('school_name').groupby(['school_name'])
school_types=school_data.set_index(["school_name"])["type"]
#print(school_types)
total_student = school_name['Student ID'].count()
per_school_count=combined_data["school_name"].value_counts()
budget_per_student = (school_data.set_index('school_name')['budget']/school_data.set_index('school_name')['size'])
average_math_score = school_name['math_score'].mean()
average_reading_score = school_name['reading_score'].mean()
pass_math_percent = combined_data[combined_data['math_score'] >= 70].groupby('school_name')['Student ID'].count()/total_student*100
pass_read_percent = combined_data[combined_data['reading_score'] >= 70].groupby('school_name')['Student ID'].count()/total_student*100
total_school_budget = school_data.set_index('school_name')['budget']
# % Overall Passing (The percentage of students that passed math and reading.

#print(per_school_count)
#per_school_budget = school_data[("school_name")("budget")].value()
overall_pass = combined_data[(combined_data['reading_score'] >= 70) & (combined_data['math_score'] >= 70)].groupby('school_name')['Student ID'].count()/total_student*100

#print(per_school_budget)
#per_school_capita = 
school_summary = pd.DataFrame({
    "School Type": school_types,
    "Total Students": total_student,
    "Per Student Budget": budget_per_student,
    "Total School Budget": total_school_budget,
    "Average Math Score": average_math_score,
    "Average Reading Score": average_reading_score,
    '% Passing Math': pass_math_percent,
    '% Passing Reading': pass_read_percent,
    "% Overall Passing": overall_pass
})
school_summary.style.format({'Total Students': '{:}',
                          "Total School Budget": "${:,.2f}",
                          "Per Student Budget": "${:.2f}",
                          'Average Math Score': "{:6f}", 
                          'Average Reading Score': "{:6f}", 
                          "% Passing Math": "{:6f}", 
                          "% Passing Reading": "{:6f}"})

,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,$628.00,"$3,124,928.00",77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,$582.00,"$1,081,356.00",83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,$639.00,"$1,884,411.00",76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,$644.00,"$1,763,916.00",77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,$625.00,"$917,500.00",83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,$652.00,"$3,022,020.00",77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,$581.00,"$248,087.00",83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,$655.00,"$1,910,635.00",76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,$650.00,"$3,094,650.00",77.072464,80.966394,66.057551,81.222432,53.539172


In [57]:
#by performance
top_perform = school_summary.sort_values("% Overall Passing", ascending = False)
top_perform.head().style.format({'Total Students': '{:}',
                           "Total School Budget": "${:,.2f}", 
                           "Per Student Budget": "${:.2f}", 
                           "% Passing Math": "{:6f}", 
                           "% Passing Reading": "{:6f}", 
                           "% Overall Passing": "{:6f}"})

,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,$582.00,"$1,081,356.00",83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,$638.00,"$1,043,130.00",83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,$625.00,"$917,500.00",83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,$578.00,"$1,319,574.00",83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,$609.00,"$585,858.00",83.839917,84.044699,94.594595,95.945946,90.540541


In [58]:
#lowerperformance
bottom_perform = top_perform.tail()
bottom_perform = bottom_perform.sort_values('% Overall Passing')
bottom_perform.style.format({'Total Students': '{: }', 
                       "Total School Budget": "${:,.2f}", 
                       "Per Student Budget": "${:.2f}", 
                       "% Passing Math": "{:6f}", 
                       "% Passing Reading": "{:6f}", 
                       "% Overall Passing": "{:6f}"})

,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,$637.00,"$2,547,363.00",76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,$639.00,"$1,884,411.00",76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,$655.00,"$1,910,635.00",76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,$652.00,"$3,022,020.00",77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,$650.00,"$3,094,650.00",77.072464,80.966394,66.057551,81.222432,53.539172


In [59]:
#grades by year and usimg loc
#creates grade level average math scores for each school 
ninth_math = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')["math_score"].mean()
tenth_math = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')["math_score"].mean()
eleventh_math = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')["math_score"].mean()
twelfth_math = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')["math_score"].mean()

math_scores = pd.DataFrame({
        "9th": ninth_math,
        "10th": tenth_math,
        "11th": eleventh_math,
        "12th": twelfth_math
})
math_scores = math_scores[['9th', '10th', '11th', '12th']]
math_scores.index.name = "School Name"

#show
math_scores.style.format({'9th': '{:.6f}', 
                          "10th": '{:.6f}', 
                          "11th": "{:.6f}", 
                          "12th": "{:.6f}"})

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [60]:
#reading
#
ninth_reading = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')["reading_score"].mean()
tenth_reading = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')["reading_score"].mean()
eleventh_reading = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')["reading_score"].mean()
twelfth_reading = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')["reading_score"].mean()

#merging
reading_scores = pd.DataFrame({
        "9th": ninth_reading,
        "10th": tenth_reading,
        "11th": eleventh_reading,
        "12th": twelfth_reading
})
reading_scores = reading_scores[['9th', '10th', '11th', '12th']]
reading_scores.index.name = "School Name"

#format
reading_scores.style.format({'9th': '{:.6f}', 
                             "10th": '{:.6f}', 
                             "11th": "{:.6f}", 
                             "12th": "{:.6f}"})

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [63]:
#spending
# bins
bins = [0, 584, 629, 644, 675]
group_name = ["<$584", "$585-629", "$630-644", "$645-675"]

combined_data['spending_bins'] = pd.cut(combined_data['budget']/combined_data['size'], bins, labels = group_name)

#groupspending by 
by_spending = combined_data.groupby('spending_bins')


avg_math = by_spending['math_score'].mean()

avg_read = by_spending['reading_score'].mean()
pass_math = combined_data[combined_data['math_score'] >= 70].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()*100
pass_read = combined_data[combined_data['reading_score'] >= 70].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()*100
overall = combined_data[(combined_data['reading_score'] >= 70) & (combined_data['math_score'] >= 70)].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()*100
    
# df             
scores_by_spend = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    "% Passing Math": pass_math,
    "% Passing Reading": pass_read,
    "% Overall Passing": overall
            
})
            
# columns
scores_by_spend = scores_by_spend[[
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing"
]]

scores_by_spend.index.name = "Per Student Budget"


#formating
scores_by_spend.style.format({'Average Math Score': '{:.2f}', 
                              'Average Reading Score': '{:.2f}', 
                              '% Passing Math': '{:.2f}', 
                              '% Passing Reading':'{:.2f}', 
                              '% Overall Passing': '{:.2f}'})

C:\Users\siddi\AppData\Local\Temp\ipykernel_25668\2836901784.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  by_spending = combined_data.groupby('spending_bins')
C:\Users\siddi\AppData\Local\Temp\ipykernel_25668\2836901784.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pass_math = combined_data[combined_data['math_score'] >= 70].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()*100
C:\Users\siddi\AppData\Local\Temp\ipykernel_25668\2836901784.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Per Student Budget,,,,,
<$584,83.36,83.96,93.70,96.69,90.64
$585-629,79.98,82.31,79.11,88.51,70.94
$630-644,77.82,81.30,70.62,82.60,58.84
$645-675,77.05,81.01,66.23,81.11,53.53


In [65]:
#Size
#  bins
bins = [0, 1000, 1999,5000]
group_name = ["Small (<1000)", "Medium (1000-2000)" , "Large (2000-5000)"]
combined_data['size_bins'] = pd.cut(combined_data['size'], bins, labels = group_name)

#group by
by_size = combined_data.groupby('size_bins')

#formulas
average_math_score = by_size['math_score'].mean()
average_reading_score = by_size['math_score'].mean()
pass_math_percent = combined_data[combined_data['math_score'] >= 70].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()*100
pass_read_percent = combined_data[combined_data['reading_score'] >= 70].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()*100
overall = combined_data[(combined_data['reading_score'] >= 70) & (combined_data['math_score'] >= 70)].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()*100

            
# df          
scores_by_size = pd.DataFrame({
    "Average Math Score": average_math_score,
    "Average Reading Score": average_reading_score,
    '% Passing Math': pass_math_percent,
    '% Passing Reading': pass_read_percent,
    '% Overall Passing': overall
            
})
            
#columns
scores_by_size = scores_by_size[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    '% Overall Passing'
]]

scores_by_size.index.name = "Total Students"
scores_by_size = scores_by_size.reindex(group_name)

#formating
scores_by_size.style.format({'Average Math Score': '{:.6f}', 
                              'Average Reading Score': '{:.6f}', 
                              '% Passing Math': '{:.6f}', 
                              '% Passing Reading':'{:.6f}', 
                              '% Overall Passing': '{:.6f}'})

C:\Users\siddi\AppData\Local\Temp\ipykernel_25668\2454410254.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  by_size = combined_data.groupby('size_bins')
C:\Users\siddi\AppData\Local\Temp\ipykernel_25668\2454410254.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pass_math_percent = combined_data[combined_data['math_score'] >= 70].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()*100
C:\Users\siddi\AppData\Local\Temp\ipykernel_25668\2454410254.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observe

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Total Students,,,,,
Small (<1000),83.828654,83.828654,93.952484,96.040317,90.136789
Medium (1000-2000),83.372682,83.372682,93.616522,96.773058,90.624267
Large (2000-5000),77.477597,77.477597,68.652380,82.125158,56.574046


In [66]:
#by school typeee
# group 
school_type = combined_data.groupby("type")

#formulas
average_math = school_type['math_score'].mean()
average_reading = school_type['math_score'].mean()
pass_math_percent = combined_data[combined_data['math_score'] >= 70].groupby('type')['Student ID'].count()/school_type['Student ID'].count()*100
pass_read_percent = combined_data[combined_data['reading_score'] >= 70].groupby('type')['Student ID'].count()/school_type['Student ID'].count()*100
overall = combined_data[(combined_data['reading_score'] >= 70) & (combined_data['math_score'] >= 70)].groupby('type')['Student ID'].count()/school_type['Student ID'].count()*100

# df build            
scores_schoo_type = pd.DataFrame({
    "Average Math Score": average_math,
    "Average Reading Score": average_reading,
    '% Passing Math': pass_math_percent,
    '% Passing Reading': pass_read_percent,
    "% Overall Passing": overall})
    
#reorder columns
scores_schoo_type = scores_schoo_type[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "% Overall Passing"
]]
scores_schoo_type.index.name = "Type of School"


#formating
scores_schoo_type.style.format({'Average Math Score': '{:.6f}', 
                              'Average Reading Score': '{:.6f}', 
                              '% Passing Math': '{:.6f}', 
                              '% Passing Reading':'{:.6f}', 
                              '% Overall Passing': '{:.6f}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Type of School,,,,,
Charter,83.406183,83.406183,93.701821,96.645891,90.560932
District,76.987026,76.987026,66.518387,80.905249,53.695878
